In [43]:
# Librerias utiles
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly
import seaborn as sns 
import plotly.express as px
import missingno as msno
import plotly.io as pio
import plotly.graph_objects as go
from mapsmx import MapsMX
import geopandas
import pyproj
import json
sns.set_theme()

In [44]:
state = MapsMX().get_geo('state')
muns = MapsMX().get_geo('municipality')
yuc = muns[muns['cve_ent']=='31']

In [45]:
victimas = pd.read_csv('reporte_semujeres_02_SEP_2021.csv',low_memory=False, dtype={'pk_perfil_agresor': 'object','num_hijos':'int64'}, parse_dates=['fecha_recepcion'])

## Casos de violencia hacia las mujeres por tipo de violencia (Estatal/Municipal)

In [46]:
victimas = pd.concat([victimas, victimas.tipos.str.get_dummies(sep=',')], axis=1)

In [50]:
tipos = victimas.iloc[:,[i for i in range(55,61)]]
tipos = pd.DataFrame(tipos.sum())
tipos = tipos.sort_values(0,ascending=False)
new_row = pd.Series(data={0:sum(victimas.descripcion_otro_tipos=='FEMINICIDA')}, name='Feminicida')
#append row to the dataframe
tipos = tipos.append(new_row)
tipos.loc['Otro',0] = tipos.loc['Otro',0] - tipos.loc['Feminicida',0]
tipos = tipos.sort_values(0, ascending=False)
y = tipos[0].values
y_total = len(victimas)
fig = px.bar(x=tipos.index,
            y=tipos[0],
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Tipo de violencia', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:7],
            color_discrete_map="identity",
            title='Gráfica 1. Tipos de violencias registradas en {}'.format('Yucatán')
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis_title=None,  height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
#pio.write_html(fig, file='../docs/figs/g7.html',include_plotlyjs='cdn', full_html=False)
#print(fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='Tipo de violencia=%{x}<br>Número de casos=%{y}<br>Porcentaje=%{text} %<extra></extra>')
fig.show()